In [ ]:
import os
from openai import OpenAI
 


client = OpenAI(
    # This is the default and can be omitted
    api_key='your API key',
)
def llm_chat(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-3.5-turbo",
    )
    return chat_completion

def gpt4_chat(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4-turbo-preview",
    )
    return chat_completion
def gpt4_6_chat(prompt):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model="gpt-4-0613",
    )
    return chat_completion

In [5]:
import json
import sys
from sem_metric.sem_metric import Squad_SEM
from datasets import load_dataset, load_metric
import datasets


metric = Squad_SEM()


/var/folders/54/vll8pfc15hq52jgzkklxt__00000gn/T/ipykernel_88312/156169343.py:8: FutureWarning: Metric is deprecated and will be removed in the next major version of datasets. Use the new library 🤗 Evaluate instead: https://huggingface.co/docs/evaluate
  metric = Squad_SEM()


In [6]:
train_data = [json.loads(line) for line in open('../Documents/tr_v5/train_joint.json')]
data = [json.loads(line) for line in open('test_gold_gcs.json')]

In [24]:
import tiktoken
encoding = tiktoken.encoding_for_model('gpt-4-turbo')
def process_context(text, length):
    return encoding.decode(encoding.encode(text)[:length])

In [52]:
example_prompt_template = 'Given context: {} Q: {} A: {}.'
question_prompt_template = 'Please answer the questions given context: {}, Q: {}, please answer the names only, if there are multiple answers, please join them by \'and\' A:'
i=1
example_prompt = example_prompt_template.format(train_data[i]['fact_context'],train_data[i]['question'], ' and '.join(train_data[i]['text_answers']['text'] ))
#example_prompt = example_prompt_template.format(process_context(train_data[i]['context'], 1024),train_data[i]['question'], ' and '.join(train_data[i]['text_answers']['text'] ))

print(example_prompt)


Given context: Henri Villat worked for Lycée Malherbe from January 1902 to January 1906. 
 Henri Villat worked for University of Caen Normandy from January 1906 to January 1911. 
 Henri Villat worked for University of Strasbourg from January 1919 to January 1925. 
 Henri Villat studied at Lycée Malherbe from January 1886 to January 1899. 
 Henri Villat worked for University of Montpellier from January 1911 to January 1919. 
 Henri Villat studied at École normale supérieure  (Paris) from January 1899 to January 1902. Q: Where was Henri Villat educated in October 1901? A: École normale supérieure  (Paris).


In [ ]:
import time 
ground_truth = []
preds = []
not_predicted = []
for example in data:
    question_prompt = question_prompt_template.format(example['fact_context'], example['question'])
    prompt = example_prompt + question_prompt
    try:
        response = llm_chat(prompt)
        prediction_text = response.choices[0].message.content
        gt = {'id':example['id'], 'answers':{'text': example['text_answers']['text']}}
        pred = {'id':example['id'], 'no_answer_probability':0,'prediction_text':prediction_text}
        ground_truth.append(gt)
        preds.append(pred)
        time.sleep(1)
    except:
        not_predicted.append(example)
        continue
print(len(preds))
scores=[]
result = metric._compute( references=ground_truth, predictions=preds)
scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
print(', '.join(scores))
    

In [53]:
ground_truth = []
preds = []
not_predicted = []
for example in data:
    question_prompt = question_prompt_template.format(example['fact_context'], example['question'])
    prompt = example_prompt + question_prompt

    response = gpt4_6_chat(prompt)
    prediction_text = response.choices[0].message.content
    gt = {'id':example['id'], 'answers':{'text': example['text_answers']['text']}}
    pred = {'id':example['id'], 'no_answer_probability':0,'prediction_text':prediction_text}
    ground_truth.append(gt)
    preds.append(pred)

print(len(preds))
scores=[]


329


In [43]:
ground_truth = []
preds = []
not_predicted = []
for example in data:
    question_prompt = question_prompt_template.format(process_context(example['context'],1024), example['question'])
    prompt = example_prompt + question_prompt

    response = gpt4_6_chat(prompt)
    prediction_text = response.choices[0].message.content
    gt = {'id':example['id'], 'answers':{'text': example['text_answers']['text']}}
    pred = {'id':example['id'], 'no_answer_probability':0,'prediction_text':prediction_text}
    ground_truth.append(gt)
    preds.append(pred)

print(len(preds))
scores=[]

329


In [54]:
scores=[]
result = metric._compute( references=ground_truth, predictions=preds)
scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
print(', '.join(scores))

58.36, 71.84, 77.51, 81.90


In [55]:
json.dump(preds, open('llm_predictions/gpt4_preds_reason.json','w'))

In [132]:
predictions_raw = json.load(open('gpt4_preds_reason.json'))
data = [json.loads(line) for line in open('test_gold_gcs.json')]
ground_truth = []
not_predicted = []
id2example = {}
for example in data:
    gt = {'id':example['id'], 'answers':{'text': example['text_answers']['text']}}
    ground_truth.append(gt)
    id2example[example['id']] =example
scores=[]
result = metric._compute( references=ground_truth, predictions=predictions_raw)
scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
print(', '.join(scores))

58.36, 71.84, 77.51, 81.90


In [133]:
unified_data = {'L2_1Target':{'predictions':[], 'ground_truth': []},
                'L2_MTarget':{'predictions':[], 'ground_truth': []},
                #'L3_1Target':{'predictions':[], 'ground_truth': []},
                #'L3_MTarget':{'predictions':[], 'ground_truth': []},
               }
unified_data_hop = {'L2_1hop':{'predictions':[], 'ground_truth': []},
                'L2_Mhop':{'predictions':[], 'ground_truth': []},
                #'L3_1hop':{'predictions':[], 'ground_truth': []},
                #'L3_Mhop':{'predictions':[], 'ground_truth': []},
               }

lvl_sets = set()
id2example = {}
for example in data:
    id2example[example['id']] = example
for i in range(len(predictions_raw)):
    pred = predictions_raw[i]
    example = id2example[pred['id']]
    elements = example['id'].split('_')
    if len(elements)==4:
        lvl, subj, rel, num = elements
    elif len(elements)==5:
        lvl, subj, rel1, rel2, num = elements
    lvl_sets.add(lvl)
    gt = {'id':example['id'], 'answers':{'text': example['text_answers']['text']}}
    ground_truth.append(gt)
    if lvl.startswith('L2') and len(example['text_answers']['text'])==1:
        unified_data['L2_1Target']['predictions'].append(pred)
        unified_data['L2_1Target']['ground_truth'].append(gt)
    if lvl.startswith('L2') and len(example['text_answers']['text'])>1:
        unified_data['L2_MTarget']['predictions'].append(pred)
        unified_data['L2_MTarget']['ground_truth'].append(gt)
    if lvl.startswith('L3') and len(example['text_answers']['text'])==1:
        unified_data['L2_1Target']['predictions'].append(pred)
        unified_data['L2_1Target']['ground_truth'].append(gt)
    if lvl.startswith('L3') and len(example['text_answers']['text'])>1:
        unified_data['L2_MTarget']['predictions'].append(pred)
        unified_data['L2_MTarget']['ground_truth'].append(gt)
    if lvl.startswith('L2'):
        if lvl == 'L2':
            unified_data_hop['L2_1hop']['predictions'].append(pred)
            unified_data_hop['L2_1hop']['ground_truth'].append(gt)
        else:
            unified_data_hop['L2_Mhop']['predictions'].append(pred)
            unified_data_hop['L2_Mhop']['ground_truth'].append(gt)
    if lvl.startswith('L3'):
        if lvl == 'L3':
            unified_data_hop['L2_1hop']['predictions'].append(pred)
            unified_data_hop['L2_1hop']['ground_truth'].append(gt)
        else:
            unified_data_hop['L2_Mhop']['predictions'].append(pred)
            unified_data_hop['L2_Mhop']['ground_truth'].append(gt)


print(lvl_sets)
sums = 0
keys = []
scores = []
for key in unified_data.keys():
    #print(key)
    preds= unified_data[key]['predictions']
    refs= unified_data[key]['ground_truth']
    result = metric._compute( references=refs, predictions=preds)
    #scores += [result['Set_Acc'],result['Answer_F1']]
    scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
    keys += [key]
    #bin2prob[(sy,ey)] = 1 - len(refs)/1643 
    sums+=len(refs)
print(keys)
print(', '.join(scores))
keys = []
#scores = []
all_preds = []
all_refs = []
for key in unified_data.keys():
    all_preds += unified_data[key]['predictions']
    all_refs += unified_data[key]['ground_truth']
result = metric._compute( references=all_refs, predictions=all_preds)
#scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
print(', '.join(scores))

sums = 0
keys = []
scores = []
for key in unified_data_hop.keys():
    #print(key)
    preds= unified_data_hop[key]['predictions']
    refs= unified_data_hop[key]['ground_truth']
    result = metric._compute( references=refs, predictions=preds)
    #print('{:.1f},{:.1f}'.format(result['Set_Acc'],result['Answer_F1']))
    #scores += [result['Set_Acc'],result['Answer_F1']]
    scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1'])]
    #scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
    keys += [key]
    #bin2prob[(sy,ey)] = 1 - len(refs)/1643 
    sums+=len(refs)
print(keys)
print(', '.join(scores))

keys = []
#scores = []
all_preds = []
all_refs = []
for key in unified_data_hop.keys():
    all_preds += unified_data_hop[key]['predictions']
    all_refs += unified_data_hop[key]['ground_truth']
result = metric._compute( references=all_refs, predictions=all_preds)
#scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1']),"{:.2f}".format(result['EM']) ,"{:.2f}".format(result['Token_F1'])]
scores += ['{:.2f}'.format(result['Set_Acc']),"{:.2f}".format(result['Answer_F1'])]
print(', '.join(scores))


{'L3', 'L2', 'L3H', 'L2M', 'L2H', 'L3M'}
['L2_1Target', 'L2_MTarget']
60.62, 71.20, 73.75, 78.99, 50.00, 74.22, 91.43, 92.67
60.62, 71.20, 73.75, 78.99, 50.00, 74.22, 91.43, 92.67, 58.36, 71.84, 77.51, 81.90
['L2_1hop', 'L2_Mhop']
67.13, 80.21, 51.61, 65.41
67.13, 80.21, 51.61, 65.41, 58.36, 71.84


In [140]:
pred_ex

{'id': 'L2_Q451225_P108_4',
 'no_answer_probability': 0,
 'prediction_text': 'Delft University of Technology and Leiden University'}

{'id': 'L2_Q451225_P108_4',
 'no_answer_probability': 0,
 'prediction_text': 'Delft University of Technology and Leiden University'}

In [157]:
key = 'L2_MTarget'
exam_preds = unified_data[key]['predictions']
exam_refs = unified_data[key]['ground_truth']
count = 0
for pred_ex, ref_ex in zip(exam_preds, exam_refs):
    p_id = pred_ex['id']
    example = id2example[p_id]
    result = metric._compute( references=[ref_ex], predictions=[pred_ex])
    if (result['Set_Acc'] == 0) and (result['Answer_F1'] > 0) and ('L3' in p_id):
        print(count+1)
        print(example['fact_context'])
        print(example['question'])
        print(example['date'])
        print(pred_ex['prediction_text'])
        print(ref_ex['answers']['text'])
        print(' ')
        count += 1
        if count >=10:
            break

1
Roy Jenkins held the position of Chancellor of the Exchequer from November 1967 to June 1970. 
 Roy Jenkins held the position of Representative of the Parliamentary Assembly of the Council of Europe from April 1956 to January 1957. 
 Roy Jenkins held the position of Member of the 46th Parliament of the United Kingdom from February 1974 to September 1974. 
 Roy Jenkins held the position of Chancellor of the University of Oxford from March 1987 to January 2003. 
 Roy Jenkins held the position of Member of the 41st Parliament of the United Kingdom from May 1955 to September 1959. 
 Roy Jenkins held the position of Member of the 44th Parliament of the United Kingdom from March 1966 to May 1970. 
 Roy Jenkins held the position of Member of the 47th Parliament of the United Kingdom from October 1974 to January 1977. 
 Roy Jenkins held the position of Member of the House of Lords from November 1987 to January 2003. 
 Roy Jenkins held the position of Member of the 43rd Parliament of the Unit

In [131]:
import numpy as np
from datetime import datetime
date_formats = {'abv_date_us':'%b %d %Y',
                'abv_date_sg':'%d %b %Y',
               'year':'%Y',
               'full_date_us':'%B %d %Y',
               'full_date_sg':'%d %B %Y',
               'abv_month':'%b %Y', }



chunk = 20
start = 1900
dlc_dir = ''
yearly_bins = {}
yearly_bins[(1000, 1900)] = {'predictions':[], 'references':[], 'examples':[],}

for i in range(7):
    sy = start + chunk * i
    ey = sy + chunk
    yearly_bins[(sy, ey)] = {'predictions':[], 'references':[],'examples':[] }


intra_year = {'predictions':[], 'references':[]}
inter_year = {'predictions':[], 'references':[]}

years = []
for pred, ref in zip(predictions_raw, ground_truth):
    #_, s_id, rel, idx = example['id'].split('_')
    idx = pred['id']
    example = id2example[idx]
    try:
        q_date = datetime.strptime(example['date'], date_formats['full_date_us'])
    except:
        print(example['date'])
    str_year = str(q_date.year)
    num_year = q_date.year
    years.append(num_year)
    if  example['fact_context'].count(str_year) >=2 :
        intra_year['predictions'].append(pred)
        intra_year['references'].append(ref)

    else:
        inter_year['predictions'].append(pred)
        inter_year['references'].append(ref) 
    flag = 0
    for sy, ey in yearly_bins.keys():
        if q_date.year in range(sy, ey):
            yearly_bins[(sy,ey)]['predictions'].append(pred)
            yearly_bins[(sy,ey)]['references'].append(ref)
            yearly_bins[(sy,ey)]['examples'].append(id2example[ref['id']])
            
            flag = 1
    if flag==0:
        print(q_date.year)
bin2prob = {}
sums = 0
for sy, ey in yearly_bins.keys():
    preds= yearly_bins[(sy,ey)]['predictions']
    refs= yearly_bins[(sy,ey)]['references']
    examples= yearly_bins[(sy,ey)]['examples']
    facts_lens = [len(x['fact_context'].split('\n')) for x in examples]
    ans_lens = [len(x['text_answers']['text']) for x in examples]
    #print(np.mean(facts_lens))
    #print(np.mean(ans_lens))
    result = metric._compute( references=refs, predictions=preds)
    #print(len(refs))
    #print('From {} to {}:'.format(sy,ey))
    print('{:.1f},{:.1f},{:.1f},{:.1f}'.format(result['Set_Acc'],result['Answer_F1'],result['EM'],  result['Token_F1'] ))
    #bin2prob[(sy,ey)] = 1 - len(refs)/1643 
    sums+=len(refs)


print(max(years))
print(min(years))

62.5,87.5,100.0,100.0
50.0,69.2,80.0,84.0
60.0,77.0,85.0,85.8
63.0,72.2,81.5,87.2
50.0,66.9,71.9,75.2
58.5,70.5,76.9,84.6
59.5,71.1,73.0,77.0
63.6,77.0,90.9,97.0
2023
1659


In [154]:
import numpy as np
from datetime import datetime
date_formats = {'abv_date_us':'%b %d %Y',
                'abv_date_sg':'%d %b %Y',
               'year':'%Y',
               'full_date_us':'%B %d %Y',
               'full_date_sg':'%d %B %Y',
               'abv_month':'%b %Y', }



chunk = 20
start = 1900
dlc_dir = ''
yearly_bins = {}
yearly_bins[(1000, 2010)] = {'predictions':[], 'references':[], 'examples':[],}
yearly_bins[(2010, 2040)] = {'predictions':[], 'references':[], 'examples':[],}




for pred, ref in zip(predictions_raw, ground_truth):
    #_, s_id, rel, idx = example['id'].split('_')
    idx = pred['id']
    example = id2example[idx]
    try:
        q_date = datetime.strptime(example['date'], date_formats['full_date_us'])
    except:
        print(example['date'])
    str_year = str(q_date.year)

    flag = 0
    for sy, ey in yearly_bins.keys():
        if q_date.year in range(sy, ey):
            yearly_bins[(sy,ey)]['predictions'].append(pred)
            yearly_bins[(sy,ey)]['references'].append(ref)
            yearly_bins[(sy,ey)]['examples'].append(id2example[ref['id']])
            
            flag = 1
    if flag==0:
        print(q_date.year)
bin2prob = {}
sums = 0
for sy, ey in yearly_bins.keys():
    preds= yearly_bins[(sy,ey)]['predictions']
    refs= yearly_bins[(sy,ey)]['references']
    examples= yearly_bins[(sy,ey)]['examples']
    facts_lens = [len(x['fact_context'].split('\n')) for x in examples]
    ans_lens = [len(x['text_answers']['text']) for x in examples]
    #print(np.mean(facts_lens))
    #print(np.mean(ans_lens))
    result = metric._compute( references=refs, predictions=preds)
    #print(len(refs))
    #print('From {} to {}:'.format(sy,ey))
    print('{:.1f},{:.1f},{:.1f},{:.1f}'.format(result['Set_Acc'],result['Answer_F1'],result['EM'],  result['Token_F1'] ))
    #bin2prob[(sy,ey)] = 1 - len(refs)/1643 
    sums+=len(refs)

56.5,71.1,78.0,82.3
64.9,74.2,75.7,80.5


In [129]:
print(len(yearly_bins[(2020, 2040)]['references']))

11
